<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import vari e inizializzazione pacchetti**

In [1]:
pip install textattack

     |████████████████████████████████| 337kB 5.2MB/s 
     |████████████████████████████████| 102kB 4.5MB/s 
     |████████████████████████████████| 163kB 6.5MB/s 
     |████████████████████████████████| 778kB 8.1MB/s 
     |████████████████████████████████| 337kB 8.0MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 1.1MB 16.3MB/s 
     |████████████████████████████████| 3.0MB 20.8MB/s 
     |████████████████████████████████| 17.7MB 273kB/s 
     |████████████████████████████████| 245kB 45.6MB/s 
     |████████████████████████████████| 19.7MB 1.5MB/s 
     |████████████████████████████████| 983kB 40.7MB/s 
     |████████████████████████████████| 798kB 35.2MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 1.1MB 34.8MB/s 
     |████████████████████████████████| 890kB 33.1MB/s 
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5587 sha256=f3091faa18d06f9a1537

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **NOTE IMPORTANTI**


> la funzione **Augmenter.augment(arg)** utilizza al suo interno **random**, peccato che non si puo settare il seed durante la fase di costruzione dell'oggetto o quando viene richiamata la funzione, questo ha comportato l'import di random per forzare il seed prima dell'esecuzione del codice.



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit # pewr il testing

# **AMAZON**


In [4]:
dataset = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Amazon/Reviews.csv')
print(f"COLONNE:{dataset.columns}")
print(f"DIMENSIONE:{dataset.shape}")

COLONNE:Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
DIMENSIONE:(568454, 10)


In [5]:
dataset.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


Filtering del dataset dei dati duplicati e rimozione parti inutili di esso

In [19]:
rar=dataset.copy()
#print(rar.groupby(['Text']).count().shape)
rar.drop_duplicates(subset ="Text", 
                     keep = "last", inplace = True) 
rar.drop(['Id','ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator','Time'], axis=1, inplace=True)
print(f"COLONNE:{rar.columns}")
print(f"DIMENSIONE:{rar.shape}")

COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(393579, 3)


In [14]:
from textattack.augmentation import WordNetAugmenter
from collections import OrderedDict
random.seed(33)

my_items = list()
number_of_trasformation_to_add=2

start_time = timeit.default_timer()
augmenter = WordNetAugmenter(pct_words_to_swap=1, transformations_per_example=number_of_trasformation_to_add)

for row in rar.head(10).itertuples():
  #print(row.Text)
  result=augmenter.augment(row.Text)
  # print(f'original:{s} \n')
  #print(f'Augmented:{result[0]} \n')
  #Aggiungo al dataset finale la entry originale del dataset
  my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':row.Text})
  for i in range(number_of_trasformation_to_add):
    #Aggiungo al dataset finale una nuova entry augmentata 
    my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':result[i]})
  

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=rar.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=12.278150671000049
COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(30, 3)


In [13]:
# import transformations, contraints, and the Augmenter
from textattack.augmentation import EmbeddingAugmenter
#Set the seed or the augmenter
random.seed(33)

my_items = list()
number_of_trasformation_to_add=2

start_time = timeit.default_timer()
augmenter = EmbeddingAugmenter(pct_words_to_swap=1, transformations_per_example=number_of_trasformation_to_add)
for row in rar.head(10).itertuples():
  #print(row.Text)
  result=augmenter.augment(row.Text)
  # print(f'original:{s} \n')
  #print(f'Augmented:{result[0]} \n')
  #Aggiungo al dataset finale la entry originale del dataset
  my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':row.Text})
  for i in range(number_of_trasformation_to_add):
    #Aggiungo al dataset finale una nuova entry augmentata 
    my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':result[i]})

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=rar.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=115.54404126700001
COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(30, 3)


In [ ]:
pd.set_option('display.max_colwidth', -1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
augmeted_new.head(3).Text

0    I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.                           
1    I have acquiring differing of the Dynamism tinned doggy meals product and have detected them all to be of alright quality. The merchandise seems more adores a simmer than a treated carnes and it olfactory best. My Labrador is fastidious and she grateful this commodities optimum than  most.
2    I have purchase myriad of the Dynamism tinned terrier catering merchandise and have discovered them all to be of alright quality. The commodity seems more loves a simmer than a process meats and it reeks optimum. My Labrador is fastidious and she grateful this commodities best than  most. 
Name: Text, dtype: object

# **ATE ABISTA**

> WORD NET AUGMENTER da fare da circa zero unsado la classe WordSwapWordNet(lang=italian or it)



In [ ]:
dataset = pd.read_json('/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/ATE_ABSITA_training_set/ate_absita_training.ndjson', lines=True)
print(f"COLONNE:{dataset.columns}")
print(f"DIMENSIONE:{dataset.shape}")

COLONNE:Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')
DIMENSIONE:(3054, 6)


In [ ]:
rar=dataset.copy()
#print(rar.groupby(['Text']).count().shape)
rar.drop_duplicates(subset ="sentence", 
                     keep = "last", inplace = True)
print(f"COLONNE:{rar.columns}")
print(f"DIMENSIONE:{rar.shape}")

COLONNE:Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')
DIMENSIONE:(3034, 6)


In [ ]:
rar.head(2)

,sentence,id_sentence,score,polarities,aspects_position,aspects
0,"Ottimo prodotto di marca, la qualità é veramen...",4b7254a1-3f31-4143-ab22-a8558aa4a73b,5,"[[0, 0], [0, 1], [1, 0]]","[[120, 142], [71, 79], [29, 36]]","[provvisto di una tasca, capiente, qualità]"
1,Ottimo rasoio dal semplice utilizzo. Rade molt...,4b74d99d-891f-4526-bbd3-549fa244cd1c,5,"[[1, 0], [1, 0], [1, 0], [1, 0]]","[[18, 26], [37, 41], [79, 86], [99, 105]]","[semplice, Rade, Pratico, pulire]"


In [ ]:
# import transformations, contraints, and the Augmenter
from textattack.transformations import WordSwapWordNet

from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter
#Set the seed or the augmenter
random.seed(33)

transformation = WordSwapWordNet(language='ita')
constraints = [RepeatModification(), StopwordModification()]# rimove la possibilità di sostituire parole gia sostituite,ed anche parole inutili dette "stopword"(the,as,a,in ecc)

my_items = list()
number_of_trasformation_to_add=2

start_time = timeit.default_timer()

for row in rar.head(6).itertuples():
  #print(row.Text)
  augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.7, transformations_per_example=number_of_trasformation_to_add)
  result=augmenter.augment(row.sentence)
  # print(f'original:{s} \n')
  #print(f'Augmented:{result[0]} \n')
  #Aggiungo al dataset finale la entry originale del dataset
  my_items.append({'sentence':row.sentence, 'id_sentence':row.id_sentence, 'score':row.score, 
                   'polarities':row.polarities, 'aspects_position':row.aspects_position,'aspects':row.aspects})
  for i in range(number_of_trasformation_to_add):
    #Aggiungo al dataset finale una nuova entry augmentata 
    my_items.append({'sentence':result[i], 'id_sentence':None, 'score':row.score, 
                   'polarities':None, 'aspects_position':None,'aspects':None})

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=rar.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")
print(f"DATI:\n{augmeted_new.head(5).sentence}")

TIME OF EXECUTION=1.1540149610000299
COLONNE:Index(['sentence', 'id_sentence', 'score', 'polarities', 'aspects_position',
       'aspects'],
      dtype='object')
DIMENSIONE:(18, 6)
DATI:
0    Ottimo prodotto di marca, la qualità é veramen...
1    Ottimo derrata di contrassegno, la natura é da...
2    Ottimo risultato di contrassegno, la nota é be...
3    Ottimo rasoio dal semplice utilizzo. Rade molt...
4    Ottimo rasoio dal merlotto uso. Rade più amore...
Name: sentence, dtype: object


Come si puo notare da questa frase italiana di esempio **WordSwapWordNet()** non è in grando di trovare nel suo db di parole italiane le varie coniugazioni dei vari verbi, il che limita la possiblità di variare il verbo a meno che esso non si all'infinito. **Questa è una grave limitazione del tool**, per analisi completa del db che crea i sinonimi visita:[link](http://compling.hss.ntu.edu.sg/omw/)

In [ ]:
# Set up transformation using CompositeTransformation()
transformation = WordSwapWordNet(language='ita')
# Set up constraints
constraints = [RepeatModification(), StopwordModification()]# rimove la possibilità di sostituire parole gia sostituite,ed anche parole inutili dette "stopword"(the,as,a,in ecc)
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=1, transformations_per_example=2)
s = 'Vorrei un cane'
# Augment!
augmenter.augment(s)

['Vorrei un incompetente', 'Vorrei un inetto']

### **TEST GENERICI DI CODICE**

In [ ]:
from nltk.corpus import wordnet
import textattack

l='eng'
word='dog'
synonyms = set()
for syn in wordnet.synsets(word, lang=l):
    for syn_word in syn.lemma_names(lang=l):
        if (
            (syn_word != word)
            and ("_" not in syn_word)
            and (textattack.shared.utils.is_one_word(syn_word))
        ):
            # WordNet can suggest phrases that are joined by '_' but we ignore phrases.
            synonyms.add(syn_word)
result=list(synonyms)

print(result)

['firedog', 'frump', 'trail', 'hound', 'blackguard', 'frank', 'cad', 'tail', 'track', 'andiron', 'hotdog', 'dog-iron', 'pawl', 'click', 'chase', 'detent', 'tag', 'wienerwurst', 'frankfurter', 'bounder', 'heel', 'wiener', 'weenie']
